# The Anatomy of a Blockbuster

## 1. Data Wrangling and EDA

This notebook preprocesses the TMDB 5000 movie dataset. The goal is to clean the data and prepare it for analysis, with a focus on understanding the relationship between a movie's budget, revenue, and genre.

### 1.1. Load the Datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
try:
    movies_df = pd.read_csv('../data/raw/tmdb_5000_movies.csv')
    credits_df = pd.read_csv('../data/raw/tmdb_5000_credits.csv')
except FileNotFoundError:
    print("Error: Dataset files not found in data/raw/. Please ensure the files are there.")
    # Create empty dataframes to avoid further errors in notebook execution
    movies_df = pd.DataFrame()
    credits_df = pd.DataFrame()

# Merge the two dataframes
if not movies_df.empty and not credits_df.empty:
    # Rename the 'id' column in movies_df to 'movie_id' to match credits_df for merging
    credits_df.rename(columns={'movie_id': 'id'}, inplace=True)
    df = pd.merge(credits_df, movies_df, on='id')
    print("Datasets loaded and merged successfully.")
    display(df.head())
else:
    print("Could not load or merge data. Please check the file paths and data integrity.")

### 1.2. Initial Data Cleaning

In this section, we perform some initial data cleaning, such as handling missing values and correcting data types.

In [ ]:
# First, let's get a sense of the missing data in the dataframe.
if 'df' in locals() and not df.empty:
    print("Missing values before cleaning:")
    display(df.isnull().sum())

    # Replace 0s with NaN in budget and revenue columns, as 0 is likely missing data
    df['budget'] = df['budget'].replace(0, np.nan)
    df['revenue'] = df['revenue'].replace(0, np.nan)

    # Drop rows where 'release_date' is missing, as it's crucial for time-based analysis
    df.dropna(subset=['release_date'], inplace=True)

    # Convert 'release_date' to datetime objects
    df['release_date'] = pd.to_datetime(df['release_date'])

    print("\nMissing values after cleaning 0s in budget/revenue and dropping rows with null release_date:")
    display(df.isnull().sum())
    
    # We will handle the remaining missing values later
else:
    print("DataFrame `df` not available. Skipping initial data cleaning.")

### 1.3. Clean and Transform Complex Columns

Several columns in the dataset are stored as JSON-like strings. We need to parse these columns to extract the relevant information. We'll focus on `genres`, `keywords`, `cast`, and `crew`.

In [ ]:
import ast

def parse_json_col(column):
    """
    Parses a JSON-like column to extract the 'name' from each object in the list.
    """
    if 'df' in locals() and not df.empty:
        # ast.literal_eval is a safe way to evaluate a string containing a Python literal
        # Here, we handle potential missing values (floats) by returning an empty list
        return column.apply(lambda x: [i['name'] for i in ast.literal_eval(x)] if isinstance(x, str) else [])
    return pd.Series() # Return empty series if df is not available

def get_director(crew):
    """
    Parses the crew column to extract the director's name.
    """
    if 'df' in locals() and not df.empty:
        if isinstance(crew, str):
            for member in ast.literal_eval(crew):
                if member['job'] == 'Director':
                    return member['name']
    return np.nan # Return NaN if no director is found

if 'df' in locals() and not df.empty:
    # Parse genres and keywords
    df['genres'] = parse_json_col(df['genres'])
    df['keywords'] = parse_json_col(df['keywords'])
    
    # Parse cast - let's take top 3 actors
    df['cast'] = df['cast'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)[:3]] if isinstance(x, str) else [])
    
    # Parse crew to get the director
    df['director'] = df['crew'].apply(get_director)

    # Now we can drop the original 'crew' column as we've extracted what we need
    df.drop(columns=['crew'], inplace=True)
    
    print("Cleaned 'genres', 'keywords', 'cast', and extracted 'director'.")
    display(df[['title', 'genres', 'keywords', 'cast', 'director']].head())
else:
    print("DataFrame `df` not available. Skipping complex column cleaning.")

### 1.4. Feature Engineering

Now that the data is cleaner, we can create new features that might be useful for our analysis. We'll extract time-based features from `release_date` and create profitability metrics.

In [ ]:
if 'df' in locals() and not df.empty:
    # Extract year, month, and day from release_date
    df['release_year'] = df['release_date'].dt.year
    df['release_month'] = df['release_date'].dt.month
    df['release_day'] = df['release_date'].dt.day
    
    # Calculate profit and return on investment (ROI)
    # We will only calculate this where both budget and revenue are available
    df['profit'] = df['revenue'] - df['budget']
    df['roi'] = (df['profit'] / df['budget']) * 100
    
    print("Created new features: 'release_year', 'release_month', 'release_day', 'profit', 'roi'.")
    display(df[['title', 'release_date', 'release_year', 'budget', 'revenue', 'profit', 'roi']].head())
else:
    print("DataFrame `df` not available. Skipping feature engineering.")

## 2. Exploratory Data Analysis (EDA)

In this section, we'll explore the cleaned dataset to uncover initial insights and answer some of our key research questions.

In [ ]:
if 'df' in locals() and not df.empty:
    print("Summary statistics for numerical columns:")
    display(df.describe())
else:
    print("DataFrame `df` not available. Skipping EDA.")

### 2.1. Budget vs. Revenue

In [ ]:
if 'df' in locals() and not df.empty:
    plt.figure(figsize=(12, 6))
    sns.scatterplot(data=df, x='budget', y='revenue', alpha=0.5)
    plt.title('Budget vs. Revenue')
    plt.xlabel('Budget (in hundred millions)')
    plt.ylabel('Revenue (in billions)')
    plt.show()
else:
    print("DataFrame `df` not available. Skipping Budget vs. Revenue plot.")

### 2.2. Genre Profitability

In [ ]:
if 'df' in locals() and not df.empty:
    # Explode the genres list to have one genre per row
    genres_df = df.explode('genres')
    
    # Calculate the median profit per genre
    genre_profit = genres_df.groupby('genres')['profit'].median().sort_values(ascending=False)
    
    plt.figure(figsize=(14, 8))
    sns.barplot(x=genre_profit.values, y=genre_profit.index, orient='h')
    plt.title('Median Profit by Genre')
    plt.xlabel('Median Profit (in hundred millions)')
    plt.ylabel('Genre')
    plt.show()
else:
    print("DataFrame `df` not available. Skipping Genre Profitability plot.")

### 2.3. Top 10 Most Profitable Movies

In [ ]:
if 'df' in locals() and not df.empty:
    # Sort by profit and get the top 10
    top_10_profitable = df.sort_values(by='profit', ascending=False).head(10)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(data=top_10_profitable, x='profit', y='title', orient='h')
    plt.title('Top 10 Most Profitable Movies')
    plt.xlabel('Profit (in billions)')
    plt.ylabel('Movie Title')
    plt.show()
else:
    print("DataFrame `df` not available. Skipping Top 10 Profitable Movies plot.")